In [1]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
import tensorflow.keras.callbacks
from tensorflow.keras import initializers

from tensorflow.keras.datasets import mnist

In [2]:
def load_data(dataset):
    if dataset == "mnist":
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
    else:
        x_train = np.load("Data/" + dataset + ".npy")
    x_train = (x_train.astype(np.float32) - 127.5)/127.5
    x_train = x_train.reshape(len(x_train), 784)
    return (x_train)

In [19]:
def build_1D_generator(rand_shape, output_shape):
    
    data_in = Input(shape=rand_shape, name="data_in")
    
    x = Dense(256, kernel_initializer=initializers.RandomNormal(stddev=0.02))(data_in)       
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Dense(512)(x)       
    x = LeakyReLU(alpha=0.2)(x)

    x = Dense(1024)(x)       
    x = LeakyReLU(alpha=0.2)(x)

    out = Dense(output_shape[0], activation='tanh')(x)
    
    generator = Model(inputs=[data_in], outputs=[out])
    generator.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])

    generator.summary()
    return generator

In [20]:
def build_1D_discriminator(input_shape):
    
    data_in = Input(shape=input_shape, name="data_in")
    
    x = Dense(1024, kernel_initializer=initializers.RandomNormal(stddev=0.02))(data_in)       
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)
    
    x = Dense(512)(x)       
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)

    x = Dense(256)(x)       
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)

    out = Dense(1, activation='sigmoid')(x)
    
    discriminator = Model(inputs=[data_in], outputs=[out])
    discriminator.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])

    discriminator.summary()
    return discriminator

# Build GAN

In [3]:
def build_gan(generator, discriminator, rand_shape):
    
    # set generation mode
    discriminator.trainable = False
    
    gan_input = Input(shape=rand_shape)
    x = generator(gan_input)
    gan_output = discriminator(x)
    
    gan = Model(inputs=[gan_input], outputs=[gan_output])
    gan.compile(loss="binary_crossentropy", optimizer="adadelta")
    
    return gan

# Train 1D GAN

In [22]:
def train_gan(dataset, rand_param_size=100, epochs=10, batch_size=32, show_img=True):
    np.random.seed(10)
    # Load data
    x_train = load_data(dataset)
    # Set shapes and stuff
    rand_shape = (rand_param_size, )
    output_shape = x_train.shape[1:]
    batch_count = int(x_train.shape[0] / batch_size)

    # Build models
    generator = build_1D_generator(rand_shape, output_shape)
    discriminator = build_1D_discriminator(output_shape)
    gan = build_gan(generator, discriminator, rand_shape)

    # Train for each epoch
    for i in range(1, epochs + 1):
        print('-'*15, 'Epoch %d' % i, '-'*15)
        # for each batch
        for j in tqdm(range(batch_count)):
            # build fake image
            noise = np.random.normal(0, 1, size=[batch_size, rand_param_size])
            generated_images = generator.predict(noise)
            # grab real images
            image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]
            # Build mixed dataset
            X = np.concatenate([image_batch, generated_images])
            # Build labels
            y_dis = np.zeros(2*batch_size)
            y_dis[:batch_size] = 0.9
            # Discriminator train step
            discriminator.trainable = True
            discriminator.train_on_batch(X, y_dis)
            # Build new seeds and "real" labels
            seeds = np.random.normal(0, 1, size=[batch_size, rand_param_size])
            y_gen = np.ones(batch_size)
            # Generator train step
            discriminator.trainable = False
            gan.train_on_batch(seeds, y_gen)
    
    if show_img:
        # build fake images
        visu_seeds = np.random.normal(0, 1, size=[batch_size, rand_param_size])
        built_images = generator.predict(visu_seeds)
        built_images = built_images.reshape(built_images.shape[0], int(np.sqrt(built_images.shape[1])), int(np.sqrt(built_images.shape[1])))
        # plot generated images
        fig = plt.figure(figsize=(8, 8))
        columns = 4
        rows = batch_size / columns
        for i, img in enumerate(built_images):
            if i + 1 > 0 and i < columns * rows:
                fig.add_subplot(rows, columns, i+1)
                plt.imshow(img, cmap="Greys")
        plt.show()

In [23]:
train_gan("mnist", rand_param_size=100, epochs=2, batch_size=32, show_img=True)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
data_in (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense (Dense)                (None, 256)               25856     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1024)              0     

  0%|          | 0/1875 [00:00<?, ?it/s]

--------------- Epoch 1 ---------------


  4%|▍         | 83/1875 [00:08<02:56, 10.13it/s]


KeyboardInterrupt: 

# 2D (WIP)

In [2]:
def load_2D_data(dataset):
    if dataset == "mnist":
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
    else:
        x_train = np.load("Data/" + dataset + ".npy")
    x_train = (x_train.astype(np.float32) - 127.5)/127.5
    x_train = np.expand_dims(x_train, axis=3)
    return (x_train)

In [23]:
def build_2D_generator(rand_shape, output_shape):
    
    depth = 64
    
    data_in = Input(shape=rand_shape, name="data_in")
    
    x = Dense(output_shape[0] * output_shape[1] * output_shape[2], 
              kernel_initializer=initializers.RandomNormal(stddev=0.02))(data_in)       
    x = LeakyReLU(alpha=0.2)(x)

    x = Reshape(output_shape)(x)
    
    x = UpSampling2D()(x)
    x = Conv2DTranspose(depth // 2, 5, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling2D()(x)
    x = Conv2DTranspose(depth // 4, 5, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(depth // 8, 5, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
     
    out = Conv2D(1, 5, strides=2, padding='same', activation="sigmoid")(x)
    
    generator = Model(inputs=[data_in], outputs=[out])
    generator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    generator.summary()
    return generator

In [24]:
def build_2D_discriminator(input_shape):
    
    data_in = Input(shape=input_shape, name="data_in")

    x = Conv2D(filters=64, kernel_size=32, strides=2,
                padding='same', activation="relu", use_bias=False, 
                kernel_initializer=initializers.RandomNormal(stddev=0.02))(data_in)       
    x = BatchNormalization()(x)

    x = Conv2D(filters=32, kernel_size=16, strides=2, padding='same', 
                      activation="relu", use_bias=False)(x)       
    x = BatchNormalization()(x)

    x = Conv2D(filters=16, kernel_size=8, strides=2, padding='same', 
                      activation="relu", use_bias=False)(x)       
    x = BatchNormalization()(x)

    x = Flatten(name='flattened')(x)

    out = Dense(1, activation='sigmoid')(x)
    
    discriminator = Model(inputs=[data_in], outputs=[out])
    discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    discriminator.summary()

    return discriminator

In [25]:
def build_2D_gan(generator, discriminator, rand_shape):
    
    # set generation mode
    discriminator.trainable = False
    
    gan_input = Input(shape=rand_shape)
    x = generator(gan_input)
    gan_output = discriminator(x)
    
    gan = Model(inputs=[gan_input], outputs=[gan_output])
    gan.compile(loss="binary_crossentropy", optimizer="adadelta")
    
    return gan

In [26]:
def train_2D_gan(dataset, rand_param_size=100, epochs=10, batch_size=32, show_img=True):
    np.random.seed(10)
    # Load data
    x_train = load_2D_data(dataset)
    # Set shapes and stuff
    rand_shape = (rand_param_size, )
    output_shape = x_train.shape[1:]
    batch_count = int(x_train.shape[0] / batch_size)

    # Build models
    generator = build_2D_generator(rand_shape, output_shape)
    discriminator = build_2D_discriminator(output_shape)
    gan = build_2D_gan(generator, discriminator, rand_shape)

    # Train for each epoch
    for i in range(1, epochs + 1):
        print('-'*15, 'Epoch %d' % i, '-'*15)
        # for each batch
        for j in tqdm(range(batch_count)):
            # build fake image
            noise = np.random.normal(0, 1, size=[batch_size, rand_param_size])
            generated_images = generator.predict(noise)
            # grab real images
            image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]
            # Build mixed dataset
            X = np.concatenate([image_batch, generated_images])
            # Build labels
            y_dis = np.zeros(2*batch_size)
            y_dis[:batch_size] = 0.9
            # Discriminator train step
            discriminator.trainable = True
            discriminator.train_on_batch(X, y_dis)
            # Build new seeds and "real" labels
            seeds = np.random.normal(0, 1, size=[batch_size, rand_param_size])
            y_gen = np.ones(batch_size)
            # Generator train step
            discriminator.trainable = False
            gan.train_on_batch(seeds, y_gen)
    
    if show_img:
        # build fake images
        visu_seeds = np.random.normal(0, 1, size=[batch_size, rand_param_size])
        built_images = generator.predict(visu_seeds)
        built_images = built_images.reshape(built_images.shape[0], int(np.sqrt(built_images.shape[1])), int(np.sqrt(built_images.shape[1])))
        # plot generated images
        fig = plt.figure(figsize=(8, 8))
        columns = 4
        rows = batch_size / columns
        for i, img in enumerate(built_images):
            if i + 1 > 0 and i < columns * rows:
                fig.add_subplot(rows, columns, i+1)
                plt.imshow(img, cmap="Greys")
        plt.show()

In [27]:
train_2D_gan("mnist", rand_param_size=100, epochs=2, batch_size=64, show_img=True)

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
data_in (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_7 (Dense)              (None, 784)               79184     
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 784)               0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
up_sampling2d_8 (UpSampling2 (None, 56, 56, 1)         0         
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 56, 56, 32)        832       
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 56, 56, 32)        0   




  0%|          | 0/937 [00:00<?, ?it/s]

--------------- Epoch 1 ---------------





  0%|          | 1/937 [00:03<59:05,  3.79s/it]


  0%|          | 2/937 [00:07<58:40,  3.77s/it]


  0%|          | 3/937 [00:11<58:45,  3.77s/it]


  0%|          | 4/937 [00:15<58:27,  3.76s/it]


  1%|          | 5/937 [00:18<57:47,  3.72s/it]


  1%|          | 6/937 [00:22<57:25,  3.70s/it]


  1%|          | 7/937 [00:25<57:17,  3.70s/it]


  1%|          | 8/937 [00:29<57:26,  3.71s/it]

KeyboardInterrupt: 